In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
from lxml import etree
import re
from datetime import datetime
import openpyxl

In [2]:
def scrape_table(table):
    data = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)
    return data

In [3]:
### S&P 500 ###
'''
Code deprecated, use selenium
'''

# url = "https://stockanalysis.com/list/sp-500-stocks/"
# response = requests.get(url)
# html_content = response.content
# soup = BeautifulSoup(html_content, "html.parser")
# table = soup.find("table", {"id":"main-table"})
# sp500_data = scrape_table(table)

'\nCode deprecated, use selenium\n'

In [3]:
### Code for test scraping with S&P500 ###
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://stockanalysis.com/list/sp-500-stocks/")

columns_dropdown_button = driver.find_element(By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/div[3]/div[2]/div[3]/button")
columns_dropdown_button.click()
sector_checkbox_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/div[3]/div[2]/div[3]/div/div[2]/div[10]/input"))
)
sector_checkbox_button.click()
time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find("table", {"id":"main-table"})
sp500_data = scrape_table(table)

In [4]:
sp500df = pd.DataFrame(sp500_data, columns=["No.", "Symbol", "Company Name", "Market Cap", "Stock Price", "% Change", "Revenue", "Sector"], )

In [5]:
#### NYSE STOCKS ####
data = []
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://stockanalysis.com/list/nyse-stocks/")
for i in range(4):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", {"id":"main-table"})
    data.extend(scrape_table(table))
    next_button = driver.find_element(By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/nav/button[2]")
    next_button.click()
    time.sleep(3)

In [ ]:
df = pd.DataFrame(data, columns=["No.", "Symbol", "Company Name", "Market Cap", "Stock Price", "% Change", "Revenue"], )

In [6]:
top_100_stock_ticker = df.iloc[:100].Symbol

In [6]:
k = 10
top_k_sp500_stock_ticker = sp500df.iloc[:k].Symbol

In [7]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)
smart_score_results = []

for stock_ticker in top_k_sp500_stock_ticker:
    try:
        driver.get(f"https://www.tipranks.com/stocks/{stock_ticker}")
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[4]"))
        )
        smart_score = re.search(r"Stock Smart Score\s*(\d+)", element.text).group(1)
        avg_price_tgt = re.search(r"Average Price Target:\s*\$(\d+\.\d+)", element.text).group(1)
        smart_score_results.append((stock_ticker, smart_score, avg_price_tgt))
    except:
        print(f"Error printing smart score for {stock_ticker}")
        continue
    
    #print(f"{stock_ticker} smart score: {smart_score}")

In [ ]:
# For debugging
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)
smart_score_results = []

driver.get(f"https://www.tipranks.com/stocks/tgt")
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[4]"))
)
smart_score = re.search(r"Stock Smart Score\s*(\d+)", element.text).group(1)
avg_price_tgt = re.search(r"Average Price Target:\s*\$(\d+\.\d+)", element.text).group(1)
smart_score

'3'

In [7]:
sorted(smart_score_results, key=lambda x: int(x[1]), reverse=True)

[('MSFT', '10', '627.67'),
 ('AMZN', '10', '264.13'),
 ('META', '10', '873.14'),
 ('AVGO', '10', '344.50'),
 ('WMT', '10', '113.86'),
 ('V', '10', '397.95'),
 ('KO', '10', '80.07'),
 ('DIS', '10', '137.00'),
 ('T', '10', '31.59'),
 ('ANET', '10', '148.27'),
 ('QCOM', '10', '180.33'),
 ('BSX', '10', '125.14'),
 ('TJX', '10', '152.75'),
 ('COF', '10', '259.20'),
 ('PANW', '10', '217.07'),
 ('COP', '10', '115.89'),
 ('WELL', '10', '184.10'),
 ('MO', '10', '61.22'),
 ('AMT', '10', '248.86'),
 ('COIN', '10', '374.43'),
 ('APO', '10', '164.67'),
 ('EMR', '10', '147.50'),
 ('NVDA', '9', '211.36'),
 ('CVX', '9', '170.67'),
 ('UNH', '9', '318.81'),
 ('AMD', '9', '184.91'),
 ('ABT', '9', '145.94'),
 ('BX', '9', '186.81'),
 ('ISRG', '9', '600.94'),
 ('SPGI', '9', '624.69'),
 ('ACN', '9', '327.27'),
 ('ETN', '9', '387.09'),
 ('APH', '9', '118.09'),
 ('UNP', '9', '260.58'),
 ('KKR', '9', '160.62'),
 ('ADP', '9', '327.00'),
 ('SHW', '9', '391.13'),
 ('WM', '9', '257.87'),
 ('PNC', '9', '213.71'),
 (

In [20]:
res = sp500df.iloc[:k]
smart_score_df = pd.DataFrame(smart_score_results, columns=["Symbol", "Smart Score", "Average Price Target"])
excel_output = pd.merge(smart_score_df, res)
excel_output.drop("No.", inplace=True, axis=1)
excel_output["Smart Score"] = excel_output["Smart Score"].astype(int)
excel_output["Average Price Target"] = excel_output["Average Price Target"].astype(float)
excel_output["Stock Price"] = excel_output["Stock Price"].astype(float)
excel_output["Upside"] = (excel_output["Average Price Target"] - excel_output["Stock Price"]) / excel_output["Stock Price"]
excel_output["Upside"] = (excel_output["Upside"] * 100).round(2).astype(str) + "%"
excel_output

,Symbol,Smart Score,Average Price Target,Company Name,Market Cap,Stock Price,% Change,Revenue,Sector,Upside
0,NVDA,9,211.36,NVIDIA Corporation,"4,058.59B",167.02,-2.70%,165.22B,Technology,26.55%
1,MSFT,10,627.67,Microsoft Corporation,"3,679.42B",495.00,-2.55%,281.72B,Technology,26.8%
2,AAPL,7,239.36,Apple Inc.,"3,557.09B",239.69,-0.04%,408.63B,Technology,-0.14%
3,GOOG,5,235.42,Alphabet Inc.,"2,843.01B",235.17,1.08%,371.40B,Communication Services,0.11%
4,GOOGL,8,231.94,Alphabet Inc.,"2,842.09B",235.00,1.16%,371.40B,Communication Services,-1.3%
5,AMZN,10,263.95,"Amazon.com, Inc.","2,477.78B",232.33,-1.42%,670.04B,Consumer Discretionary,13.61%
6,META,10,873.14,"Meta Platforms, Inc.","1,890.26B",752.45,0.51%,178.80B,Communication Services,16.04%
7,AVGO,10,367.68,Broadcom Inc.,"1,575.15B",334.89,9.41%,59.93B,Technology,9.79%
8,TSLA,7,309.00,"Tesla, Inc.","1,131.62B",350.84,3.64%,92.72B,Consumer Discretionary,-11.93%
9,BRK.B,8,538.00,Berkshire Hathaway Inc.,"1,078.40B",499.77,-1.41%,370.15B,Financials,7.65%


In [21]:
today_str = datetime.today().strftime('%Y%m%d')
excel_output.to_excel(f'scraped_scores/{today_str}_top_{k}_smart_score.xlsx')